In [12]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import time
import sys

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Python version: 3.13.9 | packaged by conda-forge | (main, Oct 16 2025, 10:23:36) [MSC v.1944 64 bit (AMD64)]
PyTorch version: 2.8.0+cu129
CUDA available: True
CUDA version: 12.9
GPU: NVIDIA GeForce RTX 5070 Ti


## 1. NLLB-200 모델 테스트
경량화된 600M 모델부터 테스트

라이센스 X

In [ ]:
# NLLB-600M 모델 로드
print("Loading NLLB-600M model...")
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# GPU 사용 가능 여부 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
model = model.to(device)
print("Model loaded successfully!")

In [3]:
def translate_ko_to_en(text: str, model, tokenizer, device="cpu"):
    """한국어를 영어로 번역"""
    inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)
    
    translated_tokens = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids("eng_Latn"),
        max_length=512,
        num_beams=5,
        early_stopping=True
    )
    
    result = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    return result

def translate_en_to_ko(text: str, model, tokenizer, device="cpu"):
    """영어를 한국어로 번역"""
    inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)
    
    translated_tokens = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids("kor_Hang"),
        max_length=512,
        num_beams=5,
        early_stopping=True
    )
    
    result = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    return result

## 2. 테스트 케이스 준비

In [4]:
# AI 에이전트 개발 관련 테스트 문장들
test_sentences_ko = [
    "AI 에이전트를 개발하는 것은 매우 흥미로운 작업입니다.",
    "Python으로 MCP 서버를 구축하고 있습니다.",
    "머신러닝 모델의 성능을 최적화하는 방법에는 여러 가지가 있습니다.",
    "한국어 프롬프트를 영어로 번역한 후 LLM에 전달합니다.",
    "번역 품질을 높이기 위해 전문화된 모델을 사용합니다.",
    "FastMCP를 사용하면 MCP 서버를 쉽게 만들 수 있습니다.",
]

test_sentences_en = [
    "Developing an AI agent is a very interesting task.",
    "I am building an MCP server with Python.",
    "There are several ways to optimize machine learning model performance.",
    "We translate Korean prompts into English and then pass them to the LLM.",
    "We use specialized models to improve translation quality.",
    "FastMCP makes it easy to create MCP servers.",
]

In [6]:
# 한국어 → 영어 번역 테스트
print("=" * 80)
print("한국어 → 영어 번역 테스트 (NLLB-600M)")
print("=" * 80)

total_time = 0
for i, ko_text in enumerate(test_sentences_ko, 1):
    start_time = time.time()
    en_text = translate_ko_to_en(ko_text, model, tokenizer, device)
    elapsed = time.time() - start_time
    total_time += elapsed
    
    print(f"\n[{i}] 원문 (한국어):")
    print(f"  {ko_text}")
    print(f"번역 (영어):")
    print(f"  {en_text}")
    print(f"소요 시간: {elapsed:.2f}초")
    print("-" * 80)

print(f"\n평균 번역 시간: {total_time / len(test_sentences_ko):.2f}초")

한국어 → 영어 번역 테스트 (NLLB-600M)

[1] 원문 (한국어):
  AI 에이전트를 개발하는 것은 매우 흥미로운 작업입니다.
번역 (영어):
  Developing an AI agent is a very interesting job.
소요 시간: 0.36초
--------------------------------------------------------------------------------

[2] 원문 (한국어):
  Python으로 MCP 서버를 구축하고 있습니다.
번역 (영어):
  I'm building an MCP server in Python.
소요 시간: 0.14초
--------------------------------------------------------------------------------

[3] 원문 (한국어):
  머신러닝 모델의 성능을 최적화하는 방법에는 여러 가지가 있습니다.
번역 (영어):
  There are many ways to optimize the performance of machine learning models.
소요 시간: 0.18초
--------------------------------------------------------------------------------

[4] 원문 (한국어):
  한국어 프롬프트를 영어로 번역한 후 LLM에 전달합니다.
번역 (영어):
  Korean Language Prompt is translated into English and then transferred to LLM.
소요 시간: 0.20초
--------------------------------------------------------------------------------

[5] 원문 (한국어):
  번역 품질을 높이기 위해 전문화된 모델을 사용합니다.
번역 (영어):
  We use specialized models to improve the quality of tr

In [7]:
# 영어 → 한국어 번역 테스트
print("=" * 80)
print("영어 → 한국어 번역 테스트 (NLLB-600M)")
print("=" * 80)

total_time = 0
for i, en_text in enumerate(test_sentences_en, 1):
    start_time = time.time()
    ko_text = translate_en_to_ko(en_text, model, tokenizer, device)
    elapsed = time.time() - start_time
    total_time += elapsed
    
    print(f"\n[{i}] 원문 (영어):")
    print(f"  {en_text}")
    print(f"번역 (한국어):")
    print(f"  {ko_text}")
    print(f"소요 시간: {elapsed:.2f}초")
    print("-" * 80)

print(f"\n평균 번역 시간: {total_time / len(test_sentences_en):.2f}초")

영어 → 한국어 번역 테스트 (NLLB-600M)

[1] 원문 (영어):
  Developing an AI agent is a very interesting task.
번역 (한국어):
  인공지능 에이전트를 개발하는 것은 매우 흥미로운 작업입니다.
소요 시간: 0.32초
--------------------------------------------------------------------------------

[2] 원문 (영어):
  I am building an MCP server with Python.
번역 (한국어):
  파이썬으로 MCP 서버를 만들고 있습니다.
소요 시간: 0.14초
--------------------------------------------------------------------------------

[3] 원문 (영어):
  There are several ways to optimize machine learning model performance.
번역 (한국어):
  머신러닝 모델의 성능을 최적화하는 몇 가지 방법이 있습니다.
소요 시간: 0.21초
--------------------------------------------------------------------------------

[4] 원문 (영어):
  We translate Korean prompts into English and then pass them to the LLM.
번역 (한국어):
  우리는 한국어 문서를 영어로 번역하고 LLM에 전달합니다.
소요 시간: 0.18초
--------------------------------------------------------------------------------

[5] 원문 (영어):
  We use specialized models to improve translation quality.
번역 (한국어):
  우리는 전문적인 모델을 사용하여 번역의 질을 향상시킵니다.
소요 시간

In [10]:
test_sentences_ko = [
    "공정 목록을 알려주세요",
    "2025년 8월1일부터 9월30일까지 생산계획을 알려주세요.",
    "작업지시 'PD202509010017'의 총 생산실적 수량을 알려주세요.",
    "'5060150100PU1' 품목의 재고가 얼마나 있는지 알려주세요.",
    "오늘 출고(출하)예정인 품목과 수량 정보를 알려주세요.",
    "'PD202504170013' 작업지시에 투입해야할 투입자재 정보를 품목코드, 품목명, 유형, BOM 사용수량, 총 투입수량 형태로 알려주세요."
]

## 3. 왕복 번역 테스트 (품질 확인)

In [11]:
print("=" * 80)
print("왕복 번역 테스트 (한국어 → 영어 → 한국어)")
print("=" * 80)

for i, original_ko in enumerate(test_sentences_ko[:6], 1):
    # 한국어 → 영어
    en_translated = translate_ko_to_en(original_ko, model, tokenizer, device)
    
    # 영어 → 한국어 (다시 번역)
    ko_back = translate_en_to_ko(en_translated, model, tokenizer, device)
    
    print(f"\n[{i}] 원본 한국어:")
    print(f"  {original_ko}")
    print(f"→ 영어 번역:")
    print(f"  {en_translated}")
    print(f"→ 다시 한국어로:")
    print(f"  {ko_back}")
    print("-" * 80)

왕복 번역 테스트 (한국어 → 영어 → 한국어)

[1] 원본 한국어:
  공정 목록을 알려주세요
→ 영어 번역:
  Please tell me the list.
→ 다시 한국어로:
  리스트를 알려주세요.
--------------------------------------------------------------------------------

[2] 원본 한국어:
  2025년 8월1일부터 9월30일까지 생산계획을 알려주세요.
→ 영어 번역:
  Please indicate the production plan from 1 August 2025 to 30 September 2025.
→ 다시 한국어로:
  2025년 8월 1일부터 2025년 9월 30일까지의 생산 계획을 알려주세요.
--------------------------------------------------------------------------------

[3] 원본 한국어:
  작업지시 'PD202509010017'의 총 생산실적 수량을 알려주세요.
→ 영어 번역:
  Please indicate the total actual production volume of 'PD202509010017'.
→ 다시 한국어로:
  "PD202509010017"의 전체 실제 생산량을 표시하십시오.
--------------------------------------------------------------------------------

[4] 원본 한국어:
  '5060150100PU1' 품목의 재고가 얼마나 있는지 알려주세요.
→ 영어 번역:
  Please tell me how much is the stock of the item '5060150100PU1'.
→ 다시 한국어로:
  "5060150100PU1" 항목의 주가가 얼마나 되는지 알려주세요.
---------------------------------------------------------------------------

## 4. 대화형 테스트

In [ ]:
# 한국어 입력 테스트
your_text = "여기에 번역하고 싶은 한국어 문장을 입력하세요."
result = translate_ko_to_en(your_text, model, tokenizer, device)
print(f"원문: {your_text}")
print(f"번역: {result}")

In [ ]:
# 영어 입력 테스트
your_english = "Enter your English sentence here to translate to Korean."
result = translate_en_to_ko(your_english, model, tokenizer, device)
print(f"Original: {your_english}")
print(f"Translation: {result}")

## 5. 모델 정보

In [9]:
# 모델 크기 및 파라미터 수
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

total_params = count_parameters(model)
print(f"모델: {model_name}")
print(f"총 파라미터 수: {total_params:,}")
print(f"모델 크기: {total_params / 1e6:.1f}M parameters")

# 메모리 사용량
if device == "cuda":
    print(f"\nGPU 메모리 사용량: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"GPU 메모리 예약량: {torch.cuda.memory_reserved() / 1e9:.2f} GB")

모델: facebook/nllb-200-distilled-600M
총 파라미터 수: 615,073,792
모델 크기: 615.1M parameters

GPU 메모리 사용량: 2.48 GB
GPU 메모리 예약량: 2.55 GB


## 6. (선택) Tower 모델 테스트
번역 전문 모델인 Tower와 비교해보고 싶다면 아래 셀을 실행하세요

In [ ]:
# Tower 모델 로드 (주석 해제하여 사용)
tower_model_name = "Unbabel/TowerInstruct-7B-v0.2"
print(f"Loading {tower_model_name}...")
tower_tokenizer = AutoTokenizer.from_pretrained(tower_model_name)
tower_model = AutoModelForSeq2SeqLM.from_pretrained(tower_model_name)
tower_model = tower_model.to(device)
print("Tower model loaded!")

# Tower 모델로 번역 테스트
test_text = "AI 에이전트 개발은 흥미롭습니다."
result = translate_ko_to_en(test_text, tower_model, tower_tokenizer, device)
print(f"원문: {test_text}")
print(f"Tower 번역: {result}")